In [1]:
import os
import pandas as pd
import glob
import seaborn as sns
import pandas as pd

In [2]:
path_input=os.path.join(os.getcwd(), '../data/input/dresden/')
path_output_weather = os.path.join(os.getcwd(), '../data/output/weather/')
path_output=os.path.join(os.getcwd(), '../data/output/')

In [3]:
df =pd.read_csv(os.path.join(path_output, "Trips.csv"),index_col=0)

In [4]:
#datetime conversion
df = df.sort_values(by=['datetime_start'], ascending=True)
df["datetime_start"] = pd.to_datetime(df["datetime_start"])
df["count"] = 1

In [5]:
#temporal features
for col in ['datetime_start']:
    df['month'] = pd.DatetimeIndex(df['datetime_start']).month
    df['booking_date'] = df.datetime_start.dt.date
    df['weekdays'] = pd.DatetimeIndex(df['datetime_start']).weekday
    df['hour'] = pd.DatetimeIndex(df['datetime_start']).hour
df["booking_date"] = pd.to_datetime(df["booking_date"])
df["hour"] = pd.to_datetime(df["hour"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567439 entries, 0 to 567438
Data columns (total 43 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   p_bike_racks_start     567439 non-null  int64         
 1   p_spot_start           567439 non-null  bool          
 2   p_booked_bikes_start   567439 non-null  int64         
 3   p_place_type_start     567439 non-null  int64         
 4   datetime_start         567439 non-null  datetime64[ns]
 5   b_number_start         567439 non-null  int64         
 6   p_uid_start            567439 non-null  int64         
 7   p_bikes_start          567439 non-null  int64         
 8   p_lat_start            567439 non-null  float64       
 9   b_electric_lock_start  567439 non-null  bool          
 10  b_bike_type_start      567439 non-null  int64         
 11  p_name_start           567439 non-null  object        
 12  p_free_racks_start     567439 non-null  int6

### How we want to get temporal resolutions
- add 4 columns for each resolution: 24, 6, 2, one hour
- they display # bookings within given time period
- i dont know how this will help with predictions, since we'd still need mean data for those ?


In [6]:
#data by timespan 24H, 1H, 4H, 12H
def setDataset(dataset, temp_resol, columnnamegroupby, functions_dic):
    newDataset = dataset.resample(temp_resol, on =columnnamegroupby).agg(functions_dic)
    return newDataset
#requested resolutions
temp_resol = ['24H', '6H', '2H', '1H']

In [7]:
setDataset(df,'24H','datetime_start',{"count":"count"})

,count
datetime_start,
2019-01-20,265
2019-01-21,529
2019-01-22,477
2019-01-23,530
2019-01-24,1092
...,...
2020-01-16,1520
2020-01-17,1033
2020-01-18,810


In [8]:
# #merge from dwd data
# df_full = pd.merge_asof(df_merged.sort_values('datetime_start'), 
#                         df_dwd, left_on='datetime_start', 
#                         right_on='MESS_DATUM', 
#                         tolerance=pd.Timedelta('30 min'),
#                         allow_exact_matches=True, direction='nearest')

In [9]:
# def add_temp_resol(dataset):
#     temp_resol = ['24H', '6H', '2H', '1H']
    
#     for i in temp_resol: 
#         data_temp = dataset
#         data_temp = data_temp.resample(temp_resol, on="datetime_start").agg({"count":"count"})
#         df_full = pd.merge_asof(dataset.sort_values('datetime_start'), 
#                          data_temp, left_on='datetime_start', 
#                          right_on='datetime_start', 
#                          tolerance=pd.Timedelta('24 hours'),
#                          allow_exact_matches=True, direction='nearest')
#     return df_full
        
        


In [10]:
dataset = df

In [11]:
#base -> Shifts the base time to calculate from by some time amount
#shifting the base to the middle value of given time interval solved the NaN problem.
#Why? Because of our timedelta, which is half the amount of time of the resampled time interval 
#--> for time interval x we want to sum up all in!
#example: for daily counts aka 24 hours the base will be 12 am, the sum will be from 00:00 am until 23:59 pm
# df_24 = dataset.resample("1D", base=0.5,on="datetime_start").agg({"count":"count"})
# df_6 = dataset.resample("6H", base=3, on="datetime_start").agg({"count":"count"})
# df_2 = dataset.resample("2H", base=1, on="datetime_start").agg({"count":"count"})
# df_1 = dataset.resample("1H", base=0.5, on="datetime_start").agg({"count":"count"})
df_24 = dataset.resample("1D",  on="datetime_start").agg({"count":"count"})
df_6 = dataset.resample("6H", on="datetime_start").agg({"count":"count"})
df_2 = dataset.resample("2H", on="datetime_start").agg({"count":"count"})
df_1 = dataset.resample("1H",  on="datetime_start").agg({"count":"count"})

In [192]:
display(df_24,df_6,df_2,df_1)
#all NaN lol

,count
datetime_start,
2019-01-20,265
2019-01-21,529
2019-01-22,477
2019-01-23,530
2019-01-24,1092
2019-01-25,485
2019-01-26,261
2019-01-27,450
2019-01-28,703


,count
datetime_start,
2019-01-20 00:00:00,47
2019-01-20 06:00:00,60
2019-01-20 12:00:00,104
2019-01-20 18:00:00,54
2019-01-21 00:00:00,25
...,...
2020-01-19 18:00:00,199
2020-01-20 00:00:00,33
2020-01-20 06:00:00,317


,count
datetime_start,
2019-01-20 00:00:00,27
2019-01-20 02:00:00,14
2019-01-20 04:00:00,6
2019-01-20 06:00:00,4
2019-01-20 08:00:00,32
...,...
2020-01-20 14:00:00,140
2020-01-20 16:00:00,184
2020-01-20 18:00:00,118


,count
datetime_start,
2019-01-20 00:00:00,19
2019-01-20 01:00:00,8
2019-01-20 02:00:00,9
2019-01-20 03:00:00,5
2019-01-20 04:00:00,3
...,...
2020-01-20 19:00:00,62
2020-01-20 20:00:00,43
2020-01-20 21:00:00,27


### merging those temporal datasets together

In [15]:
#24_sum with td of 12 hours
df_full = pd.merge_asof(dataset.sort_values('datetime_start'), 
                    df_24, left_on='booking_date', 
                    right_on='datetime_start', 
                    #tolerance=pd.Timedelta('12 hours'),
                    allow_exact_matches=True, direction='nearest')
#                    allow_exact_matches=True, direction='forward')
df_full.rename(columns={'count_x':'counter',
                          'count_y':'24_sum',}, 
                 inplace=True)
df_full.head(5)

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,...,rain_yn,sun_hour,wind_ms,trip_duration,counter,month,booking_date,weekdays,hour,24_sum
0,0,False,0,12,2019-01-20 00:00:00,93771,12095573,1,51.071262,False,...,0.0,0.0,4.3,28.0,1,1,2019-01-20,6,1970-01-01,265
1,0,True,0,0,2019-01-20 00:05:00,93576,10299640,5,51.038210,False,...,0.0,0.0,4.3,10.0,1,1,2019-01-20,6,1970-01-01,265
2,0,True,0,0,2019-01-20 00:07:00,93440,10299584,1,51.042570,True,...,0.0,0.0,4.3,26.0,1,1,2019-01-20,6,1970-01-01,265
3,0,False,0,12,2019-01-20 00:07:00,93322,12098234,1,51.041798,False,...,0.0,0.0,4.3,2.0,1,1,2019-01-20,6,1970-01-01,265
4,0,True,0,0,2019-01-20 00:07:00,93585,264575,5,51.071740,False,...,0.0,0.0,4.3,28.0,1,1,2019-01-20,6,1970-01-01,265


In [12]:
df_6

,count
datetime_start,
2019-01-20 00:00:00,47
2019-01-20 06:00:00,60
2019-01-20 12:00:00,104
2019-01-20 18:00:00,54
2019-01-21 00:00:00,25
...,...
2020-01-19 18:00:00,199
2020-01-20 00:00:00,33
2020-01-20 06:00:00,317


In [235]:
from datetime import datetime

temp = df_6[df_6.index<='2020-01-19 18:00:00']
yolo = temp[temp.index==temp.index.max()]['count']
#yolo = datetime.fromtimestamp(temp.index.max().value)
yolo[0]


199

In [16]:
from datetime import timedelta
from datetime import datetime

def merge(row,df):
    temp = df[df.index<row['datetime_start']]
    try:
        return temp[temp.index==temp.index.max()]['count'][0]        
    except: 
        print('error')
        pass
    

In [ ]:
df_full['6_sum'] = df_full.apply(lambda row:merge(row, df_6), axis=1)

In [174]:
#6_sum with td of 3 hours
df_full = pd.merge_asof(df_full.sort_values('datetime_start'), 
                    df_6, left_on='datetime_start', 
                    right_on='datetime_start', 
                    tolerance=pd.Timedelta('3 hours'),
                    allow_exact_matches=True, direction='nearest')
#                    allow_exact_matches=True, direction='forward')
df_full.rename(columns={
                          'count':'6_sum',}, 
                 inplace=True)
df_full.head(5)

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,...,sun_hour,wind_ms,trip_duration,counter,month,booking_date,weekdays,hour,24_sum,6_sum
0,0,False,0,12,2019-01-20 00:00:00,93771,12095573,1,51.071262,False,...,0.0,4.3,28.0,1,1,2019-01-20,6,0,265.0,47.0
1,0,True,0,0,2019-01-20 00:05:00,93576,10299640,5,51.038210,False,...,0.0,4.3,10.0,1,1,2019-01-20,6,0,265.0,47.0
2,0,True,0,0,2019-01-20 00:07:00,93440,10299584,1,51.042570,True,...,0.0,4.3,26.0,1,1,2019-01-20,6,0,265.0,47.0
3,0,False,0,12,2019-01-20 00:07:00,93322,12098234,1,51.041798,False,...,0.0,4.3,2.0,1,1,2019-01-20,6,0,265.0,47.0
4,0,True,0,0,2019-01-20 00:07:00,93585,264575,5,51.071740,False,...,0.0,4.3,28.0,1,1,2019-01-20,6,0,265.0,47.0


In [175]:
#2_sum with td of 1 hours
df_full = pd.merge_asof(df_full.sort_values('datetime_start'), 
                    df_2, left_on='datetime_start', 
                    right_on='datetime_start', 
                    tolerance=pd.Timedelta('1 hours'),
                    allow_exact_matches=True, direction='nearest')
#                    allow_exact_matches=True, direction='forward')
df_full.rename(columns={
                          'count':'2_sum',}, 
                 inplace=True)
df_full.head(5)

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,...,wind_ms,trip_duration,counter,month,booking_date,weekdays,hour,24_sum,6_sum,2_sum
0,0,False,0,12,2019-01-20 00:00:00,93771,12095573,1,51.071262,False,...,4.3,28.0,1,1,2019-01-20,6,0,265.0,47.0,27.0
1,0,True,0,0,2019-01-20 00:05:00,93576,10299640,5,51.038210,False,...,4.3,10.0,1,1,2019-01-20,6,0,265.0,47.0,27.0
2,0,True,0,0,2019-01-20 00:07:00,93440,10299584,1,51.042570,True,...,4.3,26.0,1,1,2019-01-20,6,0,265.0,47.0,27.0
3,0,False,0,12,2019-01-20 00:07:00,93322,12098234,1,51.041798,False,...,4.3,2.0,1,1,2019-01-20,6,0,265.0,47.0,27.0
4,0,True,0,0,2019-01-20 00:07:00,93585,264575,5,51.071740,False,...,4.3,28.0,1,1,2019-01-20,6,0,265.0,47.0,27.0


In [176]:
#1_sum with td of 0.5 hours
df_full = pd.merge_asof(df_full.sort_values('datetime_start'), 
                    df_1, left_on='datetime_start', 
                    right_on='datetime_start', 
                    tolerance=pd.Timedelta('30 minutes'),
                    allow_exact_matches=True, direction='nearest')
#                    allow_exact_matches=True, direction='forward')
df_full.rename(columns={
                          'count':'1_sum',}, 
                 inplace=True)
df_full.head(5)

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,...,trip_duration,counter,month,booking_date,weekdays,hour,24_sum,6_sum,2_sum,1_sum
0,0,False,0,12,2019-01-20 00:00:00,93771,12095573,1,51.071262,False,...,28.0,1,1,2019-01-20,6,0,265.0,47.0,27.0,19.0
1,0,True,0,0,2019-01-20 00:05:00,93576,10299640,5,51.038210,False,...,10.0,1,1,2019-01-20,6,0,265.0,47.0,27.0,19.0
2,0,True,0,0,2019-01-20 00:07:00,93440,10299584,1,51.042570,True,...,26.0,1,1,2019-01-20,6,0,265.0,47.0,27.0,19.0
3,0,False,0,12,2019-01-20 00:07:00,93322,12098234,1,51.041798,False,...,2.0,1,1,2019-01-20,6,0,265.0,47.0,27.0,19.0
4,0,True,0,0,2019-01-20 00:07:00,93585,264575,5,51.071740,False,...,28.0,1,1,2019-01-20,6,0,265.0,47.0,27.0,19.0


### dive into nans

In [198]:
#this did not work as expected
#logically, 24_sum should never be nan, while 1_sum could be nan quite often.
df_full.isna().sum()
# 24_sum                   699
# 6_sum                     75
# 2_sum                     23
# 1_sum                      3
#df_full


p_bike_racks_start        0
p_spot_start              0
p_booked_bikes_start      0
p_place_type_start        0
datetime_start            0
b_number_start            0
p_uid_start               0
p_bikes_start             0
p_lat_start               0
b_electric_lock_start     0
b_bike_type_start         0
p_name_start              0
p_free_racks_start        0
b_lock_types_start        0
p_number_start           62
p_lng_start               0
b_boardcomputer_start     0
p_maintenance_start       0
p_bike_racks_end          0
p_spot_end                0
p_booked_bikes_end        0
p_place_type_end          0
datetime_end              0
p_uid_end                 0
p_bikes_end               0
p_lat_end                 0
p_name_end                0
p_free_racks_end          0
p_number_end             78
p_lng_end                 0
p_maintenance_end         0
air_deg                   0
air_hum                   0
rain_mm                   0
rain_yn                   0
sun_hour            

In [178]:
#the merge did not work for 20th of January, 2020! 
#this might be, because no matching datetime_start was found
df = df_full

df_show = df[df['24_sum'].isnull()]
df_show = df_show[['datetime_start','24_sum','6_sum','2_sum','1_sum']]
print(df_show.to_string())

#Whats the reason? maybe merge direction 'nearest' is bad for this task? --> NOPE
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge_asof.html

# A “backward” search selects the last row in the right DataFrame --> 300.000 NaNs
# whose ‘on’ key is less than or equal to the left’s key.

# A “forward” search selects the first row in the right DataFrame --> 200.000 NaNs
# whose ‘on’ key is greater than or equal to the left’s key.

# A “nearest” search selects the row in the right DataFrame --> 700 NaNs
# whose ‘on’ key is closest in absolute distance to the left’s key

#Maybe resampling method is not entirely correct?
#https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.resample.html

            datetime_start  24_sum  6_sum  2_sum  1_sum
566740 2020-01-20 12:01:00     NaN  464.0  140.0   71.0
566741 2020-01-20 12:03:00     NaN  464.0  140.0   71.0
566742 2020-01-20 12:05:00     NaN  464.0  140.0   71.0
566743 2020-01-20 12:06:00     NaN  464.0  140.0   71.0
566744 2020-01-20 12:06:00     NaN  464.0  140.0   71.0
566745 2020-01-20 12:07:00     NaN  464.0  140.0   71.0
566746 2020-01-20 12:07:00     NaN  464.0  140.0   71.0
566747 2020-01-20 12:09:00     NaN  464.0  140.0   71.0
566748 2020-01-20 12:09:00     NaN  464.0  140.0   71.0
566749 2020-01-20 12:10:00     NaN  464.0  140.0   71.0
566750 2020-01-20 12:10:00     NaN  464.0  140.0   71.0
566751 2020-01-20 12:12:00     NaN  464.0  140.0   71.0
566752 2020-01-20 12:13:00     NaN  464.0  140.0   71.0
566753 2020-01-20 12:13:00     NaN  464.0  140.0   71.0
566754 2020-01-20 12:14:00     NaN  464.0  140.0   71.0
566755 2020-01-20 12:15:00     NaN  464.0  140.0   71.0
566756 2020-01-20 12:19:00     NaN  464.0  140.0

In [183]:
df_full[df_full['booking_date']=='2020-01-21']

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,...,trip_duration,counter,month,booking_date,weekdays,hour,24_sum,6_sum,2_sum,1_sum
